## General setup
- Create a gmail account;
- Create a GEE account ([here](https://earthengine.google.com/));
- Open a google colaboratory worksheet;
- Connect your colab with your google drive;
- Sign in to your gee account.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ePJHPLn0XMb63eiEF5FuhIKTrqcC4U4EkLIAIEtZb9E&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhJbsnrbuEBLhIMI_TMpVQJ_AHZpTJm2a2Js08XftPr7Y98U3Yxp2g

Successfully saved authorization token.


#### How to obtain the shapefile of any region of the world

- Check on ([this website](https://data.humdata.org/dataset/cameroon-administrative-boundaries));
- Open the downloaded file with QGIS;
- Split the shapefile at the desired level (regional);
- Ingest the split shapefiles in the GEE plateform.

In [ ]:
### Code here

### Display Region of Interest (Western Cameroon)
- Cameroon;
- West region in Cameroon;
- Menoua Department in the west region;
- Arrondissement of Dschang in the Menoua Department.

In [ ]:
### Code here
!pip install geemap

     |████████████████████████████████| 1.9 MB 4.3 MB/s 
     |████████████████████████████████| 130 kB 50.1 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 93 kB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 61.9 MB/s 
     |████████████████████████████████| 1.2 MB 68.2 MB/s 
     |████████████████████████████████| 8.6 MB 25.8 MB/s 
     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 216 kB 58.6 MB/s 
     |████████████████████████████████| 1.7 MB 67.2 MB/s 
     |████████████████████████████████| 98 kB 6.7 MB/s 
     |████████████████████████████████| 219 kB 49.0 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 428 kB 39.1 MB/s 
     |████████████████████████████████| 396 kB 63.3 MB/s 
     |████████████████████████████████| 61 kB 3.8 MB/s 
     |█████████████████████

In [ ]:
import geemap
import ee
Map = geemap.Map(center=(4.061536, 9.786072), zoom=4, lite_mode=True) 
Map.add_basemap('HYBRID')
from geemap.basemaps import basemaps
Map.add_basemap(basemaps.OpenTopoMap)
cmr = ee.FeatureCollection("projects/earthengine-legacy/assets/users/noecareme/cmr_admbnda_adm0_inc_20180104")
Map.addLayer(cmr, {}, "Cameroon")
Map

Map(center=[4.061536, 9.786072], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [ ]:
cmr = ee.FeatureCollection("projects/earthengine-legacy/assets/users/noecareme/ADM1_PCODE_CM002")
Map.addLayer(cmr, {}, "centre")
Map

Map(bottom=2302.0, center=[4.061536, 9.786072], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

In [ ]:
cmr = ee.FeatureCollection("projects/earthengine-legacy/assets/users/noecareme/ADM1_PCODE_CM002")
Map.addLayer(cmr, {}, "Adamaoua")
Map

Map(bottom=2302.0, center=[4.061536, 9.786072], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

In [ ]:
import pandas as pd
IDSS = pd.read_csv('/content/drive/MyDrive/MODIS_DATA/WARD_ID.csv')
IDSS = list(IDSS['WARD_ID'])
col_names = [int(i) for i in IDSS]

In [ ]:
HRSL       = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrslpop")
HRSL_men   = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrsl_men")
HRSL_women = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrsl_women")
HRSL_youth = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrsl_youth")
HRSL_children_under_five    = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrsl_children_under_five")
HRSL_women_reproductive_age = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrsl_women_reproductive_age")
HRSL_elderly_over_sixty     = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrsl_elderly_over_sixty")

In [ ]:
w = pd.read_csv('/content/drive/MyDrive/MODIS_DATA/WARD_ID.csv')
w = list(w['WARD_ID'])
w[5]

52205001

In [ ]:
table = ee.FeatureCollection("users/noecareme/MODIS_WARD_10103001_20011")
kzn = cmr#table

# HRSL

HRSL1 = HRSL.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL2 = ee.Number(HRSL1.get('b1'))
except (Exception):
  HRSL2 = 0

try:
  HRSL3 = ee.Feature(None, {'HRSL': HRSL2})
except (Exception):
  HRSL3 = ee.Feature(None, {'HRSL': 0})

# HRSL_men

HRSL_men1 = HRSL_men.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL_men2 = ee.Number(HRSL_men1.get('b1'))
except (Exception):
  HRSL_men2 = 0

try:
  HRSL_men3 = ee.Feature(None, {'HRSL_men': HRSL_men2})
except (Exception):
  HRSL_men3 = ee.Feature(None, {'HRSL_men': 0})

# HRSL_women

HRSL_women1 = HRSL_women.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL_women2 = ee.Number(HRSL_women1.get('b1'))
except (Exception):
  HRSL_women2 = 0

try:
  HRSL_women3 = ee.Feature(None, {'HRSL_women': HRSL_women2})
except (Exception):
  HRSL_women3 = ee.Feature(None, {'HRSL_women': 0})

# HRSL_youth

HRSL_youth1 = HRSL_youth.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL_youth2 = ee.Number(HRSL_youth1.get('b1'))
except (Exception):
  HRSL_youth2 = 0

try:
  HRSL_youth3 = ee.Feature(None, {'HRSL_youth': HRSL_youth2})
except (Exception):
  HRSL_youth3 = ee.Feature(None, {'HRSL_youth': 0})


# HRSL_children_under_five

HRSL_children_under_five1 = HRSL_children_under_five.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL_children_under_five2 = ee.Number(HRSL_children_under_five1.get('b1'))
except (Exception):
  HRSL_children_under_five2 = 0

try:
  HRSL_children_under_five3 = ee.Feature(None, {'HRSL_children_under_five': HRSL_children_under_five2})
except (Exception):
  HRSL_children_under_five3 = ee.Feature(None, {'HRSL_children_under_five': 0})


# HRSL_women_reproductive_age

HRSL_women_reproductive_age1 = HRSL_women_reproductive_age.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL_women_reproductive_age2 = ee.Number(HRSL_women_reproductive_age1.get('b1'))
except (Exception):
  HRSL_women_reproductive_age2 = 0

try:
  HRSL_women_reproductive_age3 = ee.Feature(None, {'HRSL_women_reproductive_age': HRSL_women_reproductive_age2})
except (Exception):
  HRSL_women_reproductive_age3 = ee.Feature(None, {'HRSL_women_reproductive_age': 0})

# HRSL_elderly_over_sixty

HRSL_elderly_over_sixty1 = HRSL_elderly_over_sixty.mean().reduceRegion(**{
  'geometry': kzn.first().geometry(),
  'reducer': ee.Reducer.mean(),
  'scale': 20,
  })
try:
  HRSL_elderly_over_sixty2 = ee.Number(HRSL_elderly_over_sixty1.get('b1'))
except (Exception):
  HRSL_elderly_over_sixty2 = 0

try:
  HRSL_elderly_over_sixty3 = ee.Feature(None, {'HRSL_elderly_over_sixty': HRSL_elderly_over_sixty2})
except (Exception):
  HRSL_elderly_over_sixty3 = ee.Feature(None, {'HRSL_elderly_over_sixty': 0})

try:
  HRSL4 = ee.Feature(None, {'HRSL': HRSL3,'HRSL_men':HRSL_men3,'HRSL_women':HRSL_women3,'HRSL_youth':HRSL_youth3,'HRSL_children_under_five':HRSL_children_under_five3,'HRSL_women_reproductive_age':HRSL_women_reproductive_age3,'HRSL_elderly_over_sixty':HRSL_elderly_over_sixty3})
except (Exception):
  HRSL4 = ee.Feature(None, {'HRSL': 0,'HRSL_men':0 ,'HRSL_women':0,'HRSL_youth':0,'HRSL_children_under_five':0,'HRSL_women_reproductive_age':0,'HRSL_elderly_over_sixty':0})  
else:
  HRSL4 = ee.Feature(None, {'HRSL': 0,'HRSL_men':0 ,'HRSL_women':0,'HRSL_youth':0,'HRSL_children_under_five':0,'HRSL_women_reproductive_age':0,'HRSL_elderly_over_sixty':0})

# Export the FeatureCollection.
taskx = ee.batch.Export.table.toDrive(**{
'collection': ee.FeatureCollection([HRSL4]),
'folder': 'MEAN_HRSL',
'description': 'test3',
'fileFormat': 'CSV'
})
taskx.start()

In [ ]:
HRSL3.getInfo()